In [1]:
import pandas as pd
import numpy as np

import json
import pandas as pd
from sklearn.tree import DecisionTreeRegressor

In [2]:
train_data = pd.read_csv('train_modified.csv') # Important!!
print(train_data.shape)
train_data.head()

/Users/nomi/Documents/virtualenvs/data_analysais_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (3,50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(903653, 56)


,Unnamed: 0,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,...,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,...,NaN,NaN,NaN,(not set),NaN,NaN,google + online,organic,NaN,google
4,4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,...,NaN,NaN,NaN,(not set),NaN,True,(not provided),organic,NaN,google


In [3]:
#Print all the column names
print(train_data.columns.values)

['Unnamed: 0' 'channelGrouping' 'date' 'fullVisitorId' 'sessionId'
 'socialEngagementType' 'visitId' 'visitNumber' 'visitStartTime'
 'device.browser' 'device.browserSize' 'device.browserVersion'
 'device.deviceCategory' 'device.flashVersion' 'device.isMobile'
 'device.language' 'device.mobileDeviceBranding' 'device.mobileDeviceInfo'
 'device.mobileDeviceMarketingName' 'device.mobileDeviceModel'
 'device.mobileInputSelector' 'device.operatingSystem'
 'device.operatingSystemVersion' 'device.screenColors'
 'device.screenResolution' 'geoNetwork.city' 'geoNetwork.cityId'
 'geoNetwork.continent' 'geoNetwork.country' 'geoNetwork.latitude'
 'geoNetwork.longitude' 'geoNetwork.metro' 'geoNetwork.networkDomain'
 'geoNetwork.networkLocation' 'geoNetwork.region'
 'geoNetwork.subContinent' 'totals.bounces' 'totals.hits'
 'totals.newVisits' 'totals.pageviews' 'totals.transactionRevenue'
 'totals.visits' 'trafficSource.adContent'
 'trafficSource.adwordsClickInfo.adNetworkType'
 'trafficSource.adwordsC

In [33]:
from sklearn import preprocessing

relevantAttributes = ['totals.hits', 'visitNumber', 'totals.transactionRevenue']

trainData_DecisionTree = train_data.sample(frac=0.8,random_state=200)
testData_DecisionTree = train_data.drop(trainData_DecisionTree.index)

trainData_DecisionTree = trainData_DecisionTree[relevantAttributes]
testData_DecisionTree = testData_DecisionTree[relevantAttributes]

trainData_DecisionTree['totals.transactionRevenue'] = trainData_DecisionTree['totals.transactionRevenue'].fillna(0)
testData_DecisionTree['totals.transactionRevenue'] = testData_DecisionTree['totals.transactionRevenue'].fillna(0)

trainData_DecisionTree = pd.DataFrame(trainData_DecisionTree)
testData_DecisionTree = pd.DataFrame(testData_DecisionTree)

y = trainData_DecisionTree["totals.transactionRevenue"]

# leCountry = preprocessing.LabelEncoder()
# leCountry.fit(train_data['geoNetwork.country'])

X = trainData_DecisionTree[['totals.hits', 'visitNumber']]
# X['geoNetwork.country'] = leCountry.transform(X['geoNetwork.country'])

regressor = DecisionTreeRegressor(max_depth=5)
regressor.fit(X,y)

DecisionTreeRegressor(criterion='mse', max_depth=5, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [36]:
y = testData_DecisionTree["totals.transactionRevenue"]

X = testData_DecisionTree[['totals.hits', 'visitNumber']]
# X['geoNetwork.country'] = leCountry.transform(X['geoNetwork.country'])

y_pred = regressor.predict(X)
df=pd.DataFrame({'Actual':y, 'Predicted':y_pred})  

# regressor.score(X, y, sample_weight=None)

In [37]:
from graphviz import Source
from sklearn import tree
from IPython.display import SVG

dot_data = tree.export_graphviz(regressor,
                                feature_names=['geoNetwork.country', 'visitNumber'],
                                out_file="tree.dot",
                                filled=True,
                                rounded=True)
# SVG(graph.pipe(format='svg'))

In [50]:
y_pred = y_pred
y = y + 1

y_pred_ln = np.log(y_pred)
y_true_ln = np.log(y)

difference = np.subtract(y_pred_ln, y_true_ln)

sqr = np.square(difference)

sumSqr = sqr.sum()

sumSqrMean = sumSqr/len(y)

sumSqrRootMean = np.sqrt(sumSqrMean)
sumSqrRootMean

7.554435450589206